In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os, os.path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import xlwings as xw

In [4]:
def calc_new_build(all_states,data_MW):
    # based on capacity expansion and retirement
    ret_data = pd.read_excel(r"../../data/ReEDS/EIA_NEMS_retirement_from_ReEDS.xlsx")

    for this_state in all_states:
        # retirement schedule 
        ret_schedule = ret_data.query("State == @this_state")
        #ret_schedule.loc[:,"tech"] = ret_schedule.loc[:,'ReEDS_tech'].astype(str) + "_MW"
        ret_schedule['tech'] = ret_schedule['ReEDS_tech'].apply(lambda x: "{}{}".format(x,'_MW'))
        ret_schedule = ret_schedule.drop(columns=["State","ReEDS_tech"]).set_index("tech")
        ret_schedule = ret_schedule.sort_index(axis=1)
        ret_schedule = ret_schedule.transpose()
        ret_schedule = -ret_schedule
        ret_schedule["type"] = "retire"

        # match the retirement schedule to the format of the ReEDS capacity 
        model_cap = data_MW.query("state == @this_state")
        model_cap = model_cap.drop(columns = "state")
        model_cap_ret = model_cap.copy()
        model_cap_ret["type"] = "base"
        for year in model_cap_ret.index[1:]:
            retire_year = ret_schedule.loc[[year,year-1],:]
            model_cap_ret = model_cap_ret.append(retire_year)
        ret_years = model_cap_ret.query("type == 'retire'").drop(columns="type")
        ret_years = ret_years.transpose().replace(np.nan, 0)

        # sum the odd number years with the even number year after
        for y in ret_years.columns:
            if (y%2) != 0:
                ret_years[y+1] = ret_years[y]+ret_years[y+1]
                ret_years = ret_years.drop(columns = y)

        # calculate the number of new builds 
        model_cap_t = model_cap.transpose()
        model_cap_newbuild = model_cap_t.copy()
        for year in model_cap.index[1:]:
            model_cap_newbuild[year] =  model_cap_t[year] - model_cap_t[year-2] + ret_years[year] 
        model_cap_newbuild[2022]=0
        # assume that if new build is negative, it's actually zero
        model_cap_newbuild[model_cap_newbuild < 0] = 0 
        model_cap_newbuild = model_cap_newbuild.transpose()
        model_cap_newbuild["state"] = this_state

        # assume battery lifetime to be 14 year (actually 15 years)
        # if something is built in 
        # all other technologies have lifetime longer than the model time horizon 
        list_batteries = ["battery-10h_MW","battery-2h_MW","battery-4h_MW","battery-6h_MW","battery-8h_MW","pv+battery_MW"]
        for bat in list_batteries:
            for bat_year in model_cap_newbuild.index[:8]: # ends at 2036
                this_bat_val = model_cap_newbuild.loc[bat_year,bat]
                if this_bat_val == 0:
                    continue
                else:
                    model_cap_newbuild.loc[bat_year+14,bat] = model_cap_newbuild.loc[bat_year+14,bat] + this_bat_val

        # compile everything together
        if this_state == all_states[0]:
            all_newbuild = model_cap_newbuild
        else:
            all_newbuild = all_newbuild.append(model_cap_newbuild)
            
    return all_newbuild